In [1]:
#read data

import pandas as pd
train_file_path = "project_2_train/project_2_train/data 2_train.csv"
train_data = pd.read_csv(train_file_path)
train_data.head()

,example_id,text,aspect_term,term_location,class
0,3121_0,But the staff was so horrible to us.,staff,8--13,-1
1,2777_0,To be completely fair[comma] the only redeemin...,food,57--61,1
2,1634_0,The food is uniformly exceptional[comma] with ...,food,4--8,1
3,1634_1,The food is uniformly exceptional[comma] with ...,kitchen,55--62,1
4,1634_2,The food is uniformly exceptional[comma] with ...,menu,141--145,0


In [2]:
aspect_term = train_data[' aspect_term'].value_counts(dropna=False)
print(aspect_term)

food                                             345
service                                          197
place                                             59
prices                                            58
dinner                                            55
menu                                              55
staff                                             54
atmosphere                                        46
pizza                                             43
table                                             41
meal                                              39
price                                             38
sushi                                             35
drinks                                            33
bar                                               32
lunch                                             28
dishes                                            28
Service                                           27
portions                                      

In [3]:
class_value = train_data[' class'].value_counts(dropna=False)
print(class_value)

 1    2164
-1     805
 0     633
Name:  class, dtype: int64


In [4]:
X = train_data.drop([" class","example_id"," text"],axis=1)
X.head()

,aspect_term,term_location
0,staff,8--13
1,food,57--61
2,food,4--8
3,kitchen,55--62
4,menu,141--145


In [5]:
X.shape

(3602, 2)

In [6]:
## target data
Y = train_data[" class"]
Y.head()

0   -1
1    1
2    1
3    1
4    0
Name:  class, dtype: int64

In [7]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_bag = count_vect.fit_transform(train_data[" text"])
X_train_bag.shape

(3602, 3716)

In [8]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bag)
X_train_tfidf.shape
X_tfidf = pd.DataFrame(X_train_tfidf.toarray())

In [9]:

df2 = pd.merge(X,X_tfidf, left_index=True, right_index=True)

In [10]:
df2.head()

,aspect_term,term_location,0,1,2,3,4,5,6,7,...,3706,3707,3708,3709,3710,3711,3712,3713,3714,3715
0,staff,8--13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,food,57--61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,food,4--8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,kitchen,55--62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,menu,141--145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
##train validation data split

from sklearn.model_selection import train_test_split

train_X, val_X, train_Y, val_Y = train_test_split(df2, Y,random_state = 0)

In [11]:
one_hot_encoded_training_predictors = pd.get_dummies(df2)
#one_hot_encoded_val_predictors = pd.get_dummies(val_X)
#final_train, final_val = one_hot_encoded_training_predictors.align(one_hot_encoded_val_predictors,
#                                                                    join='left', 
#                                                                    axis=1)

In [13]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer()
imputed_X_train = my_imputer.fit_transform(one_hot_encoded_training_predictors)
#imputed_X_val = my_imputer.transform(final_val)
#print(imputed_X_train.shape)
#print(imputed_X_val.shape)

In [44]:
#Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(imputed_X_train, Y)

In [15]:
#Naive Bayes Prediction
import numpy as np
from sklearn.metrics import accuracy_score
predicted = clf.predict(imputed_X_val)
accuracy_score(val_Y, predicted)

0.647912885662432

In [17]:
#Naive Bayes using cross validation and
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import MultinomialNB


clf = MultinomialNB()
predicted = cross_val_predict(clf, imputed_X_train, Y, cv=10)
conf_mat_naive = confusion_matrix(Y,predicted,labels=[-1,0,1])
print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))

For positive class
Precision
0.6727400140154169
Recall
0.8864265927977839
F1 Score
0.7649402390438247
For negative class
Precision
0.4562899786780384
Recall
0.265179677819083
F1 Score
0.335423197492163
For neutral class
Precision
0.4336917562724014
Recall
0.19055118110236222
F1 Score
0.2647702407002188
None
(array([0.45628998, 0.43369176, 0.67274001]), array([0.26583851, 0.19115324, 0.88724584]), array([0.33594976, 0.26535088, 0.76524512]), array([ 805,  633, 2164]))


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(Y, predicted)

0.6260410882842865

In [19]:
#Random Forest train
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier()
forest_model.fit(imputed_X_train,train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
#Random Forest predict
forest_pred = forest_model.predict(imputed_X_val)
accuracy_score(val_Y, forest_pred)

0.6751361161524501

In [20]:
#Random Forest using cross validation and confusion matrix
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier()
predicted = cross_val_predict(forest_model, imputed_X_train, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
calclulateF1(conf_mat_forest)
print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))

For positive class
Precision
0.6571332436069987
Recall
0.9016620498614959
F1 Score
0.7602179836512262
For negative class
Precision
0.40302267002518893
Recall
0.1982651796778191
F1 Score
0.26578073089701
For neutral class
Precision
0.3562231759656652
Recall
0.13070866141732285
F1 Score
0.1912442396313364
For positive class
Precision
0.6727400140154169
Recall
0.8864265927977839
F1 Score
0.7649402390438247
For negative class
Precision
0.4562899786780384
Recall
0.265179677819083
F1 Score
0.335423197492163
For neutral class
Precision
0.4336917562724014
Recall
0.19055118110236222
F1 Score
0.2647702407002188
None
(array([0.40302267, 0.35622318, 0.65713324]), array([0.19875776, 0.13112164, 0.90249538]), array([0.26622296, 0.19168591, 0.76051402]), array([ 805,  633, 2164]))


In [21]:
accuracy_score(Y, predicted)

0.6096612992781788

In [16]:
def calclulateF1(conf_matrix):
    print("For positive class")
    tp_positive = conf_matrix[2,2]
    fn_positive = sum(conf_matrix[2,:], 2)-conf_matrix[2, 2]
    fp_positive = sum(conf_matrix[:,2])-conf_matrix[2, 2]
    print("Precision")
    p_positive = tp_positive/(tp_positive+fp_positive)
    print(p_positive)
    print("Recall")
    r_positive =tp_positive/(tp_positive+fn_positive)
    print(r_positive)
    print("F1 Score")
    f1_positive = (2*p_positive*r_positive)/(p_positive+r_positive)
    print(f1_positive)
    print("For negative class")
    tp_negative = conf_matrix[0,0]
    fn_negative = sum(conf_matrix[0,:], 2)-conf_matrix[0, 0]
    fp_negative = sum(conf_matrix[:,0])-conf_matrix[0, 0]
    print("Precision")
    p_negative = tp_negative/(tp_negative+fp_negative)
    print(p_negative)
    print("Recall")
    r_negative =tp_negative/(tp_negative+fn_negative)
    print(r_negative)
    print("F1 Score")
    f1_negative = (2*p_negative*r_negative)/(p_negative+r_negative)
    print(f1_negative)
    print("For neutral class")
    tp_neutral = conf_matrix[1,1]
    fn_neutral = sum(conf_matrix[1,:], 2)-conf_matrix[1, 1]
    fp_neutral = sum(conf_matrix[:,1])-conf_matrix[1, 1]
    print("Precision")
    p_neutral = tp_neutral/(tp_neutral+fp_neutral)
    print(p_neutral)
    print("Recall")
    r_neutral =tp_neutral/(tp_neutral+fn_neutral)
    print(r_neutral)
    print("F1 Score")
    f1_neutral = (2*p_neutral*r_neutral)/(p_neutral+r_neutral)
    print(f1_neutral)

In [23]:
from sklearn.ensemble import BaggingClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = BaggingClassifier()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = MLPClassifier()

#vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
#vote_model.fit(imputed_X_train,train_Y)
#vote_pred = vote_model.predict(imputed_X_val)
#accuracy_score(val_Y, vote_pred)

In [25]:
#Random Forest using cross validation and confusion matrix
from sklearn.ensemble import BaggingClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
predicted = cross_val_predict(vote_model, imputed_X_train, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

For positive class
Precision
0.6727400140154169
Recall
0.8864265927977839
F1 Score
0.7649402390438247
For negative class
Precision
0.4562899786780384
Recall
0.265179677819083
F1 Score
0.335423197492163
For neutral class
Precision
0.4336917562724014
Recall
0.19055118110236222
F1 Score
0.2647702407002188
None
(array([0.46680942, 0.35779817, 0.67226603]), array([0.27080745, 0.12322275, 0.90619224]), array([0.3427673 , 0.18331375, 0.7718953 ]), array([ 805,  633, 2164]))


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [27]:
accuracy_score(Y, predicted)

0.6265963353692393